<h1 align='center'>Алгоритмы анализа данных</h1>

<h1 align='center'>Домашнее задание № 3</h1>

<h2 align='left'>Классификация. Логистическая регрессия</h2>

In [1]:
import numpy as np

# <p style="background-color:lightgreen;font-family:newtimeroman;color:#662e2e;font-size:130%;text-align:center;border-radius:30px 30px;">Задача 1</p>

__1*.__ Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

**Решение:**

In [2]:
def calc_logloss(y, y_pred):   
    # err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    # ИЗМЕНЕНИЯ
    # добавляем условия
    err = - np.mean(y * np.log(y_pred, where=(y_pred!=0)) + (1.0 - y) * np.log(1.0 - y_pred, where=(1-y_pred!=0)))    
    # ИЗМЕНЕНИЯ    
    return err

In [3]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [4]:
# Плохой пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0, 0.2])
calc_logloss(y1, y_pred1)

-0.28842822434289517

In [5]:
# Плохой пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.2])
calc_logloss(y1, y_pred1)

0.11157177565710485

# <p style="background-color:lightgreen;font-family:newtimeroman;color:#662e2e;font-size:130%;text-align:center;border-radius:30px 30px;">Задача 2</p>

__2.__ Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

**Решение:**

In [6]:
# исходные данные

X_st = np.array([[ 1.        ,  1.        , -0.97958969,  1.        ],
                 [ 1.        ,  1.        , -0.56713087,  1.        ],
                 [ 1.        ,  2.        , -0.46401617,  2.        ],
                 [ 1.        ,  5.        , -0.77336028,  1.        ],
                 [ 1.        ,  3.        ,  0.97958969,  2.        ],
                 [ 1.        ,  0.        , -0.36090146,  1.        ],
                 [ 1.        ,  5.        ,  1.08270439,  3.        ],
                 [ 1.        , 10.        ,  2.11385144,  3.        ],
                 [ 1.        ,  1.        , -1.08270439,  1.        ],
                 [ 1.        ,  2.        ,  0.05155735,  2.        ]])

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [7]:
'''
Исходная функция:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

Внес изменения в количество аргументов. Теперь они не задаются в ручную, а вшиты в саму функцию списками.
По аналогии с заданием из первой домашки, добавил перебор по количеству итераций и скорости обучения с выводом минимального значения logloss.
При желании, можно раскрыть промежуточные результаты, но это доооолго.
'''
#
def eval_model(X, y):
    np.random.seed(42)    
    n = X.shape[0]
    etas = [1e-1, 5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4]
    n_iters = [10, 100, 250, 500, 800, 1000, 5000, 50000]
    
    min_err = np.inf
    best_params = {}
    
    for eta in etas:
        for n_iter in n_iters:
            W = np.random.randn(X.shape[1])
            cur_eta = eta
            
            for i in range(n_iter):
                z = np.dot(X, W)
                y_pred = sigmoid(z)
                err = calc_logloss(y, y_pred)

                dQ = 1/n * X.T @ (y_pred - y)
                W -= eta * dQ
                # if i % (n_iter / 10) == 0:
                #     print(f'Iteration #{i}: W = {W}, eta = {eta}, Logloss = {err}')
            if err < min_err:
                min_err = err
                best_params = {
                    'eta': eta,
                    'n_iter': n_iter
                }
    print(f'Минимальное значение logloss {round(min_err, 5)} достигается с параметрами {best_params}')
    return W

In [8]:
# W = eval_model(X_st, y, iterations=50000, eta=0.1)
W = eval_model(X_st, y)

Минимальное значение logloss 0.14351 достигается с параметрами {'eta': 0.1, 'n_iter': 50000}


# <p style="background-color:lightgreen;font-family:newtimeroman;color:#662e2e;font-size:130%;text-align:center;border-radius:30px 30px;">Задача 3</p>

__3.__ Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

**Решение:**

In [9]:
def calc_pred_proba(X, W):    
    y_pred_proba = 1 / (1 + np.exp(-(np.dot(X, W))))
    return y_pred_proba

In [10]:
calc_pred_proba(X_st, W)

array([0.55915241, 0.68452888, 0.50903676, 0.19429942, 0.8074177 ,
       0.82131179, 0.55297013, 0.29825929, 0.52584886, 0.66981026])

# <p style="background-color:lightgreen;font-family:newtimeroman;color:#662e2e;font-size:130%;text-align:center;border-radius:30px 30px;">Задача 4</p>

__4.__ Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

**Решение:**

In [11]:
def calc_pred(X, W, threshold):
    y_pred = 1 / (1 + np.exp(-(np.dot(X, W))))
    y_pred[y_pred < threshold] = 0
    y_pred[y_pred >= threshold] = 1
    
    return y_pred

In [12]:
y_pred = calc_pred(X_st, W, 0.5)
y_pred

array([1., 1., 1., 0., 1., 1., 1., 0., 1., 1.])

# <p style="background-color:lightgreen;font-family:newtimeroman;color:#662e2e;font-size:130%;text-align:center;border-radius:30px 30px;">Задача 5</p>

__5*.__ Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

**Решение:**

In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

__Accuracy__

In [14]:
accuracy_score(y, y_pred)

0.7

In [15]:
len(y_pred) == len(y)

True

In [16]:
# accuracy

def accuracy_(y, y_pred):
    if len(y_pred) == len(y):               
        true = len(y_pred[y_pred == y]) # количество раз, когда модель сделала верное предсказание
        return true / len(y_pred)
    else:
        print(f'Некорректные исходные данные')

accuracy_(y, y_pred)

0.7

__Confusion Matrix__

In [17]:
confusion_matrix(y, y_pred)

array([[2, 3],
       [0, 5]])

In [18]:
def confusion_matrix_(y, y_pred):
    TP = len(y_pred[(y_pred == y) & (y == 1)])
    TN = len(y_pred[(y_pred == y) & (y == 0)])
    FP = len(y_pred[(y_pred != y) & (y == 0)])
    FN = len(y_pred[(y_pred != y) & (y == 1)])
    data = {'y_pred = 1': [TP, FP], 'y_pred = 0': [FN, TN]}
    cm = pd.DataFrame.from_dict(data, orient='index', columns=['y = 1', 'y = 0'])
    return cm    

In [19]:
confusion_matrix_(y, y_pred)

,y = 1,y = 0
y_pred = 1,5,3
y_pred = 0,0,2


__Precision_score (Точность)__

In [20]:
precision_score(y, y_pred)

0.625

In [21]:
def precision_score_(y, y_pred):
    TP = len(y_pred[(y_pred == y) & (y == 1)])
    FP = len(y_pred[(y_pred != y) & (y == 0)])
    ps = TP / (TP + FP)
    return ps

In [22]:
precision_score_(y, y_pred)

0.625

__Recall_score (Полнота)__

In [23]:
recall_score(y, y_pred)

1.0

In [24]:
def recall_score_(y, y_pred):
    TP = len(y_pred[(y_pred == y) & (y == 1)])
    FN = len(y_pred[(y_pred != y) & (y == 1)])
    rs = TP / (TP + FN)
    return rs

In [25]:
recall_score_(y, y_pred)

1.0

__F1 мера__

In [26]:
f1_score(y, y_pred)

0.7692307692307693

In [27]:
def f1_score_(y, y_pred):
    TP = len(y_pred[(y_pred == y) & (y == 1)])
    FP = len(y_pred[(y_pred != y) & (y == 0)])
    FN = len(y_pred[(y_pred != y) & (y == 1)])
    fs = 2 * (TP / (2 * TP + FP + FN))
    return fs


In [28]:
f1_score_(y, y_pred)

0.7692307692307693